In [1]:
import pyperclip,re,pyautogui,time
import pandas as pd
from random import choice
import jieba.analyse
import time

pyautogui.PAUSE = 0.102

In [2]:
urllist = pd.read_csv('listt.csv',header=None)[0].to_list()

In [3]:
diyichuangkou = (146,9)
urlxy = (382,52)

name1 = (820,298)
name2 = (1325,298)

blankk = (300,400)
xiayiye0 = (1262,510)
xiayiye = (1262,550)

leijipingjia = (685,95)

In [4]:
def ppaste(astr):
    pyperclip.copy(astr)
    pyautogui.hotkey('ctrl', 'v')
    
def clickk(atuple):
    pyautogui.moveTo(atuple,duration=0.101,pause=0.101)
    pyautogui.click(pause=0.101)

def find_nextpage():
    pyautogui.hotkey('ctrl','f',pause=0.15)
    pyautogui.press('enter',pause=0.15)
    pyautogui.press('esc',pause=0.15)
    clickk(blankk)

def find_name():
    pyautogui.hotkey('ctrl','u',pause=0.5)
    pyautogui.hotkey('ctrl','a',pause=0.5)
    pyautogui.hotkey('ctrl','c',pause=0.5)
    sour = pyperclip.paste()
    pyautogui.hotkey('ctrl','w',pause=0.5)
    res_sou = re.findall(r'<meta name="keywords" content=".*?"/>',sour,flags=re.DOTALL)[0][31:-3]
    return res_sou

def find_from_source(asource):
    asource = asource.replace('\r\n','')
    pinglun1 = pd.Series(re.findall(r'有内容按默.{10,10000}?（匿',asource,flags=re.DOTALL))
    pinglun1 = pinglun1.map(lambda x:x[6:])
    pinglun2 = pd.Series(re.findall(r'名）.{10,10000}?（匿',asource,flags=re.DOTALL))
    pinglun = pd.concat([pinglun1,pinglun2])        

    pinglun = pinglun.str.replace('.{5,5}解释：.*','')
    pinglun = pinglun.str.replace('.*此用户没有填写评论!','')
    pinglun = pinglun.str.replace('收货.*天后追加：','。')
    pinglun = pinglun.str.replace('超级会员|名）|.{5,5}（匿','')
    pinglun = pinglun.str.replace('[0-9][0-9].{1,1}[0-9][0-9]','')
    pinglun = pinglun.str.replace('颜色分类：.*','')
    pinglun = pinglun[pinglun.map(lambda x: False if '下一页' in x else True)]
    pinglun = pinglun[pinglun.map(lambda x: False if '查看更多的' in x else True)]
    return pinglun

In [5]:
center = (940,545)
yz1 = (815,600)
yz2 = (1107,602)
def yanzheng():
    pyautogui.click(center,duration=1,pause=1)
    pyautogui.press('pagedown')
    clickk(yz1)
    pyautogui.dragTo(yz2,duration=1)

In [6]:
for num,i in enumerate(urllist):
    num = num + 1
    df = pd.Series([i,])
    skuurl = i
    assert 'detail.tmall' in skuurl
    
    #更改 URL 链接
    pyautogui.click(diyichuangkou,pause=0.5)
    pyautogui.click(urlxy,pause=0.5)
    pyautogui.hotkey('ctrl','a',pause=0.5)
    ppaste(skuurl)
    pyautogui.press('enter',pause=0.5)
    for m in range(10):
        pyautogui.press('pageup')
   
    #从页面中提取到商品名称，filename，washto-rightname
    filename = find_name()
    filename = filename.replace('*','').replace('\/','').replace('\'','')
    print(filename)
   
    #点击到累计评价，把搜索词更改为‘下一页’
    for m in range(5):
        pyautogui.press('pagedown')
    pyautogui.click(leijipingjia,duration=1)
    
    pyautogui.hotkey('ctrl','f')
    ppaste('下一页')
    pyautogui.press('enter')
    pyautogui.press('esc')
    pyautogui.click(blankk,duration=1)
    
    #判断需要循环的页面数
    pyautogui.hotkey('ctrl','a',pause=1)
    pyautogui.hotkey('ctrl','c',pause=1)
    zhengwen = pyperclip.paste()
    pagenum = int(re.findall('累计评价 [0-9].*',zhengwen)[0][5:-1])
    pagehash = pagenum // 20
    pagehash = 120 if pagehash>120 else pagehash
    
    for n in range(pagehash):
        clickk(blankk)
        pyautogui.hotkey('ctrl','a',pause=0.2)
        pyautogui.hotkey('ctrl','c',pause=0.2)
        zhengwen = pyperclip.paste()
    
        try:
            pinglun = find_from_source(zhengwen)
            df = pd.concat([df,pinglun])
        except:
            raise TypeError
    
        if '为什么被折叠' in zhengwen:
            break
        
        if pagenum < 20:
            break
        
        if '访问验证' in zhengwen:
            yanzheng()
            for m in range(5):
                pyautogui.press('pagedown')
            clickk(leijipingjia)

        find_nextpage()
        clickk(xiayiye0)
        clickk(xiayiye)
        
    rizhipath = str(num)+'_'+filename + '.xlsx'
    pd.DataFrame([]).to_excel(rizhipath)
    excelWriter = pd.ExcelWriter(rizhipath)
    
    df.drop_duplicates(inplace=True)
    df.to_excel(excelWriter,index=False,encoding='gb18030',sheet_name='源数据',header=None)
    gjc=[]
    for x in df.map(jieba.analyse.textrank):
        for y in x:
            gjc.append(y)
    pd.Series(gjc).value_counts().to_excel(excelWriter,sheet_name='源数据词频统计')
    
    excelWriter.save()
    excelWriter.close()

Skechers斯凯奇男童鞋新款动感闪灯鞋 透气魔术贴运动鞋 90585L


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\cheng.lu\AppData\Local\Temp\jieba.cache
Loading model cost 0.559 seconds.
Prefix dict has been built succesfully.


Skechers斯凯奇男童鞋新款魔术贴炫酷闪灯鞋 大童运动鞋 90710L
Skechers斯凯奇2019春夏新款男童缓震大童鞋 闪灯运动鞋 90587L
Skechers斯凯奇男童鞋闪灯大童鞋 板鞋高帮运动鞋8661LT
Skechers斯凯奇男童2019春夏新款闪灯鞋 儿童气垫运动鞋 90596L
Skechers斯凯奇男童新款绑带款炫酷闪灯鞋 时尚休闲运动鞋 90730L
Skechers斯凯奇女童鞋 圆头高帮休闲鞋 翅膀闪灯帆布鞋10758L
Skechers斯凯奇女童鞋 魔术贴闪灯印花透气休闲帆布鞋 10839L
Skechers斯凯奇女童鞋 魔术贴时尚卡通闪灯透气耐磨帆布鞋10859L
Skechers斯凯奇闪灯发光女童帆布鞋  儿童休闲鞋10728L
Skechers斯凯奇女童多彩闪钻鞋 可爱印花闪灯大童运动鞋 10925L
Skechers斯凯奇2019新款春季官方女童公主 女孩闪灯帆布鞋 10988L
Skechers斯凯奇童鞋2019春夏新款 女童闪灯鞋休闲帆布鞋 20051L
Skechers斯凯奇中大童闪灯休闲鞋 女童缓震运动鞋10767L
Skechers斯凯奇女童运动鞋 2017新款超轻缓震休闲跑步鞋10833L
Skechers斯凯奇女童鞋大童闪灯鞋 多彩气垫运动鞋 20059L
Skechers斯凯奇春季网面可爱闪灯鞋女童轻质防滑运动鞋 10920L
Skechers斯凯奇女童鞋新款可爱闪灯大童鞋 可充电运动鞋20100L
Skechers斯凯奇童鞋新款魔术贴闪灯鞋卡通印花女童运动鞋 20178L
Skechers斯凯奇童鞋新款魔术贴休闲鞋 卡通印花运动鞋 20180L
